In [92]:
%pylab
# Imports
import numpy as np
import matplotlib.pyplot as plot
#import sys
#print sys.path
#comment next line to have plot windows outside browser
#%matplotlib inline

legScale = 95.8738e-6 # 16 bit to radian
vref = 3.3 # for voltage conversion
vdivide = 3.7/2.7  # for battery scaling
vgain = 15.0/47.0  # gain of differential amplifier
RMotor = 3.3   # resistance for SS7-3.3 ** need to check **
Kt = 1.41 #  motor toriqe constant mN-m/A  ** SS7-3.3 **

#acelerometer scale in mpu6000.c set to +- 8g
# +- 32768 data
xlScale = (1/4096.0) * 9.81

# gyro in mpu6000.c scale set to +-2000 degrees per second
# +- 32768
gyroScale = (1/16.384) * (np.pi/180.0)  

length = 16
width = 8


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [93]:
# Insert filename here
#filename = './2014.02.03_17.47.54_trial_imudata.txt'
#filename = './2014.02.03_17.53.20_trial_imudata.txt'
#filename = './2014.02.03_17.54.06_trial_imudata.txt'
#filename = '.\power-test.txt'
#filename = '../turner-ip2.5/Python/Data/imudata.txt'
#filename = 'c:/users/ronf/research/development/turner-ip2.5/Python/Data/imudata.txt'
#filename = 'c:/users/ronf/research/development/telemetryData/imudata16Feb15.txt'
#filename = 'c:/users/ronf/research/development/telemetryData/left1000right1000.txt'
#filename = '../telemetryData/tripod300l290r-air.txt'
#filename = '../telemetryData/Jul15-RollSteer/200ms130degl1.txt'
filename = '../turner-ip2.5/Python/Data/imudata.txt'
#filename= '../turner-ip2.5/Python/Data/2014.12.03_12.45.27_trialAir_1Hz_JL_imudata.txt'  
print filename

../turner-ip2.5/Python/Data/imudata.txt


In [94]:
# Read in the file
f = open(filename)
lines = f.readlines()
f.close()

# Time and Date stamps of trial
dateStamp = lines[0].split()[3]
timeStamp = lines[0].split()[4][:-1]
print 'date, time:', dateStamp, timeStamp

# Stride Frequency R,L
rightStrideFreq = float(lines[1].split()[5][:-1])
leftStrideFreq = float(lines[2].split()[5][:-1])

# Phase (fractional)
phaseFractional = float(lines[3].split()[4][:-1])

# Script that generated data
script = lines[4].split()[1]

# Motor Gains
motorGains = map(float, lines[5][lines[5].find('[')+1:lines[5].find(']')].replace(',','').split())
print 'motorGains =', motorGains
# test data read
#print 'lines[8:15]', lines[8:15]

# Data splitting
columnHeaders = lines[7].replace("\"% ","").replace(" \"\r\n","").split(' | ')
columns = len(columnHeaders)
rows = len(lines)-8
# unix: data = np.reshape(map(float,"".join(lines[8:]).replace('\r\n',',')[:-1].split(',')),(rows,columns))
# windows
data = np.reshape(map(float,"".join(lines[8:]).replace('\n',',')[:-1].split(',')),(rows,columns))

#print 'first 10 lines of data[]\n', data[0:10]
# Timestamps
time = data[:,0]/1000.0

# Leg Position R,L
#rightLegPos = (data[:,1]*legScale)% (2.0*np.pi)
#leftLegPos = (data[:,2]*legScale) % (2.0*np.pi)
rightLegPos = (data[:,1]*legScale)
leftLegPos = (data[:,2]*legScale) 

# Commanded Leg Position R,L
#commandedRightLegPos = (data[:,3]*legScale) % (2.0*np.pi)
#commandedLeftLegPos = (data[:,4]*legScale) % (2.0*np.pi)
commandedRightLegPos = (data[:,3]*legScale) 
commandedLeftLegPos = (data[:,4]*legScale)

# Duty Cycle R,L
#DCR = -data[:,5]/4000.0
#DCL = -data[:,6]/4000.0
#DCR[DCR < -4000.0] = -4000.0
#DCL[DCL < -4000.0] = -4000.0
# raw 12 bit PWM value. Max = 3900.
# convert to per cent
DCR = 100.0 * data[:,5]/4096.0
DCL = 100.0 * data[:,6]/4096.0

# Gyro X,Y,Z
GyroX = data[:,7]*gyroScale
GyroY = data[:,8]*gyroScale
GyroZ = data[:,9]*gyroScale

# Accelerometer X,Y,Z
AX = data[:,10]*xlScale
AY = data[:,11]*xlScale
AZ = data[:,12]*xlScale

# BackEMF R,L
# A/D data is 10 bits, Vref+ = AVdd = 3.3V, VRef- = AVss = 0.0V
# BEMF volts = (15K)/(47K) * Vm + vref/2 - pidObjs[i].inputOffset
#RBEMF = -data[:,13]*vdivide*vref/1023.0
#LBEMF = -data[:,14]*vdivide*vref/1023.0
RBEMF = data[:,13]*vref/1024.0/vgain  # scale A/D to 0 to 3.3 V range and undo diff amp gain
LBEMF = data[:,14]*vref/1024/vgain

# Battery Voltage in volts
VBatt = data[:,15]*vdivide*vref/1023.0

#Power calculations
# i_m = (VBatt - BEMF)/R
# V_m is just VBatt
# using motor duty cycle as 0-100%
CurrentR = (np.abs(DCR)/100.0)*(np.sign(DCR)*VBatt - RBEMF)/RMotor # i_m_avg = i_m x duty cycle
CurrentL = (np.abs(DCL)/100.0)*(np.sign(DCL)*VBatt - LBEMF)/RMotor # i_m_avg = i_m x duty cycle

# torque calculation
TorqueR = Kt * CurrentR # \Tau = Kt i_m_avg
TorqueL = Kt * CurrentL # \Tau = Kt i_m_avg
    

PowerR = np.abs(VBatt* CurrentR) # P = V_m i_m_avg
PowerL = np.abs(VBatt* CurrentL) # P = V_m i_m_avg

Energy = np.zeros(len(VBatt))
#energy calculation, account that some time samples may be missing
# dt = (time[1] - time[0]) / 1000.0 # time in seconds
for i in range(1,len(VBatt)):
    Energy[i] = Energy[i-1] + (PowerR[i] + PowerL[i]) * (time[i]-time[i-1])/1000.0

#approximate integral of angle
AngleZ = np.zeros(len(GyroZ))
AngleY = np.zeros(len(GyroY))
for i in range(1,len(GyroZ)):
    AngleZ[i] = AngleZ[i-1] + GyroZ[i]/1000.0
    AngleY[i] = AngleY[i-1] + GyroY[i]/1000.0

date, time: 2016/3/30 18:59:2
motorGains = [0.0, 0.0, 300.0, 0.0, -700.0, 0.0, 0.0, 300.0, 0.0, 500.0]


In [102]:
# calculate turn we get as function of phase
#from turnCalc import turnCalc
#turnCalc(leftLegPos, rightLegPos, AngleZ)
fig = plot.figure(figsize = (length, width/2))
#leg angle vs gyro angle
plot.subplot(1,3,1)
plot.xlabel('right leg angle [rad]')
# Total Angle in radians
plot.plot(-rightLegPos,AngleZ, 'g') # gyro angle
#plot.plot(-rightLegPos,leftLegPos)
plot.plot(-rightLegPos, rightLegPos+leftLegPos,'b') # difference in leg angles
plot.plot(-commandedRightLegPos, commandedRightLegPos+commandedLeftLegPos,'g--') # difference in ref leg angles
plot.ylabel('Angle diff (rad)')
plot.legend(['Gz','leg phase','cmd phase'])
ax = fig.add_subplot(1,2,1)
ax.axhline(linewidth=1, color='m')
xticks=np.linspace(0,11,12)*np.pi
ax.set_xticks(np.round(xticks,1))
ax.xaxis.grid() #vertical lines

maxAngle = np.max(-rightLegPos)  # right leg is negative sign
print 'rightLegPos', rightLegPos
print 'maxAngle=', maxAngle
maxCycle = np.int(maxAngle/np.pi)  # half steps
print 'maxCycle=', maxCycle
startCycle=np.linspace(0,maxCycle,maxCycle+1)*np.pi  # start of next half step
phases = np.zeros(maxCycle+2)
phases[0] = rightLegPos[0]+leftLegPos[0]  # assumed initial phase difference
yawAngle = np.zeros(maxCycle+2)
yawAngle[0]= AngleZ[0]
yawChange = np.zeros(maxCycle+2)

print 'startCycle=', startCycle
j = 0
for i in range(0,len(rightLegPos)):
    if -rightLegPos[i] > startCycle[j]:
#        print 'i,j,angle', i,j,-rightLegPos[i],
        j=j+1
        phases[j] = rightLegPos[i]+leftLegPos[i]  #  phase difference at start of half step
        yawAngle[j] = AngleZ[i]  # yaw angle at start of step
        yawChange[j-1] = yawAngle[j] - yawAngle[j-1] 
#        print 'phases, yaw angle, yaw change', phases[j], yawAngle[j], yawChange[j-1]
        if j> maxCycle:
            break       
#print 'phases,yaw angle,yaw change', phases,'\n', yawAngle,'\n', yawChange
        
        
plot.subplot(1,3,2)
plot.xlabel('$\Delta \Theta$ [rad]')
# Total Angle in radians
plot.plot(rightLegPos+leftLegPos,AngleZ, 'g') # gyro angle vs phase difference
#plot.plot(phases,yawAngle,'o')
plot.plot(phases[1:maxCycle+1],yawChange[1:maxCycle+1],'-k^')
plot.ylabel('Angle Est.(rad)')
plot.legend(['Gz','$\Delta \Theta_z$'])
ax = fig.add_subplot(1,2,2)
ax.axhline(linewidth=1, color='m')
xticks=np.linspace(0,12,13)*np.pi/18  # hard coded 10 degree advance per step
ax.set_xticks(np.round(xticks,1))
ax.xaxis.grid() #vertical lines

plot.subplot(1,3,3)
plot.xlabel('right leg [rad]')
# Total Angle in radians
plot.plot(rightLegPos,leftLegPos, 'g') # phase plot between legs
#plot.plot(phases,yawAngle,'o')
plot.plot(phases[1:maxCycle+1],yawChange[1:maxCycle+1],'-k^')
plot.ylabel('left leg [rad]')
plot.legend(['Gz','$\Delta \Theta_z$'])
ax = fig.add_subplot(1,2,2)
ax.axhline(linewidth=1, color='m')
xticks=np.linspace(0,12,13)*np.pi/18  # hard coded 10 degree advance per step
ax.set_xticks(np.round(xticks,1))
ax.xaxis.grid() #vertical lines



rightLegPos [  7.66990400e-04   1.53398080e-03  -7.66990400e-04 ...,  -6.73314027e+01
  -6.73471261e+01  -6.73609319e+01]
maxAngle= 67.36093188
maxCycle= 21
startCycle= [  0.           3.14159265   6.28318531   9.42477796  12.56637061
  15.70796327  18.84955592  21.99114858  25.13274123  28.27433388
  31.41592654  34.55751919  37.69911184  40.8407045   43.98229715
  47.1238898   50.26548246  53.40707511  56.54866776  59.69026042
  62.83185307  65.97344573]


In [96]:
fig = figure(figsize = (length,width))
min=300
max=6000

# gyro data
plot.subplot(3,2,1)
plot.plot(time,GyroX,'k--')
plot.plot(time,GyroY, 'g.')
plot.plot(time,GyroZ, 'b')
xlabel('time [ms]')
ylabel('Gyro rad/s')
legend(['X', 'Y', 'Z'])

# actual and commanded leg position
plot.subplot(3,2,2)
plot.plot(time[min:max],rightLegPos[min:max]% (2.0*np.pi),'k')
plot.plot(time[min:max],leftLegPos[min:max]% (2.0*np.pi),'b')
plot.plot(time[min:max],commandedRightLegPos[min:max]% (2.0*np.pi), 'k--')
plot.plot(time[min:max],commandedLeftLegPos[min:max]% (2.0*np.pi), 'b-.')
xlabel('time [ms]')
ylabel('Leg Position')
legend(['RPos','LPos','Rref','Lref'])
#plot.plot(time[0:1000],10.0*RBEMF[0:1000],'k')
#plot.plot(time[0:1000],LBEMF[0:1000],'b')
#xlabel('time [ms]')
#ylabel('Left Back EMF (V)')
#legend(['Right', 'Left'])
ax = fig.add_subplot(3,2,2)
ax.axhline(linewidth=1, color='m')

# accelerometer data
plot.subplot(3,2,3)
plot.plot(time[min:max],AX[min:max],'k--')
plot.plot(time[min:max],AY[min:max],'g.')
plot.plot(time[min:max],AZ[min:max],'b')
xlabel('time [ms]')
ylabel('Accel $ m s^{-2}$')
legend(['X', 'Y', 'Z'])

#back EMF
plot.subplot(3,2,4)
plot.plot(time[min:max],RBEMF[min:max],'g')
plot.plot(time[min:max],LBEMF[min:max],'b')
xlabel('time [ms]')
ylabel('Back EMF (V)')
legend(['Right', 'Left'])
ax = fig.add_subplot(3,2,4)
ax.axhline(linewidth=1, color='m')

# Motor PWM 
plot.subplot(3,2,5)
plot.plot(time,DCR,'k')
plot.plot(time,DCL,'b--')
xlabel('time [ms]')
ylabel('Duty Cycle (%)')
legend(['Right', 'Left'])
ax = fig.add_subplot(3,2,5)
ax.axhline(linewidth=1, color='m')

#battery voltage
plot.subplot(3,2,6)
plot.plot(time[0:max],VBatt[0:max])
xlabel('time [ms]')
ylabel('Battery Voltage (V)')

In [97]:
# calculate instant power
fig = figure(figsize = (length,width))


# Motor PWM 
#plot.subplot(3,2,1)
#plot.plot(time,DCR,'k')
#plot.plot(time,DCL,'b--')
#xlabel('time [ms]')
#ylabel('Duty Cycle (%)')
#legend(['Right', 'Left'])
#ax = fig.add_subplot(3,2,1)
#ax.axhline(linewidth=1, color='m')

#back EMF
plot.subplot(3,2,1)
plot.plot(time,RBEMF,'k')
plot.plot(time,LBEMF,'b')
xlabel('time [ms]')
ylabel('Back EMF (V)')
legend(['Right', 'Left'])
ax = fig.add_subplot(3,2,1)
ax.axhline(linewidth=1, color='m')

#Torque
plot.subplot(3,2,3)
plot.plot(time,TorqueR,'k')
plot.plot(time,TorqueL,'b--')
plot.plot(time,TorqueR + TorqueL, 'g.')
xlabel('time [ms]')
ylabel('Torque (mN-m)')
legend(['Right', 'Left', 'R+L'])
ax = fig.add_subplot(3,2,3)
ax.axhline(linewidth=1, color='m')

# actual and commanded leg position
min=250 
max=np.size(rightLegPos)
plot.subplot(3,2,2)
xlabel('time [ms]')
# actual and commanded leg position
plot.plot(time[min:max],rightLegPos[min:max]% (2.0*np.pi),'k')
plot.plot(time[min:max],leftLegPos[min:max]% (2.0*np.pi),'b')
plot.plot(time[min:max],commandedRightLegPos[min:max]% (2.0*np.pi), 'k--')
plot.plot(time[min:max],commandedLeftLegPos[min:max]% (2.0*np.pi), 'b-.')
ylabel('Leg Position')
legend(['RPos','LPos','Rref','Lref'],bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
yticks=np.linspace(0,2,3)*np.pi
ax = fig.add_subplot(3,2,2)
ax.set_yticks(np.round(yticks,2))
ax.yaxis.grid() #horiz lines

#plot.plot(time,commandedRightLegPos- rightLegPos,'k')
#plot.plot(time,commandedLeftLegPos - leftLegPos,'b')
#plot.plot(time,commandedRightLegPos, 'k--')
#plot.plot(time,commandedLeftLegPos, 'b-.')

#ylabel('Leg Position error')
#legend(['RErr', 'LErr'])
#battery voltage
#plot.subplot(3,2,4)
#plot.plot(time,VBatt)
#xlabel('time [ms]')
#ylabel('Battery Voltage (V)')


# Motor Power 
plot.subplot(3,2,5)
xlabel('time [ms]')
#plot.plot(time,PowerR,'k')
#plot.plot(time,PowerL,'b--')
#ylabel('battery power (W)')
#legend(['Right', 'Left'])
plot.plot(time,commandedRightLegPos- rightLegPos,'k')
plot.plot(time,commandedLeftLegPos - leftLegPos,'b')
ylabel('Leg Position error')
legend(['RErr', 'LErr'])
ax = fig.add_subplot(3,2,5)
ax.axhline(linewidth=1, color='m')




plot.subplot(3,2,4)
xlabel('time [ms]')
# Total Angle in radians
plot.plot(time,AngleY,'k')
plot.plot(time,AngleZ, 'g')
ylabel('Angle (rad)')
legend(['Gy', 'Gz'],bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax = fig.add_subplot(3,2,4)
ax.axhline(linewidth=1, color='m')

#leg angle vs gyro angle
#plot.subplot(3,2,6)
#xlabel('right leg angle [rad]')
# Total Angle in radians
#plot.plot(-rightLegPos,AngleY,'k')
#plot.plot(-rightLegPos,AngleZ, 'g')
#ylabel('Angle Est.(rad)')
#legend(['Gy', 'Gz'])
#ax = fig.add_subplot(3,2,6)
#ax.axhline(linewidth=1, color='m')
#xticks=np.linspace(0,11,12)*np.pi
#ax.set_xticks(np.round(xticks,1))
#ax.xaxis.grid() #vertical lines

plot.subplot(3,2,6)
plot.xlabel('leg phase $\Delta \Theta$ [rad]')
# Total Angle in radians
plot.plot(rightLegPos+leftLegPos,AngleZ, 'g') # gyro angle vs phase difference
#plot.plot(phases,yawAngle,'o')
plot.plot(phases[1:maxCycle+1],yawChange[1:maxCycle+1],'-k^')
plot.ylabel('Angle Est.(rad)')
plot.legend(['Gz','$\Delta \Theta_z$'])
plot.legend(['Gz','$\Delta \Theta_z$'], bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax = fig.add_subplot(3,2,6)
ax.axhline(linewidth=1, color='m')
xticks=np.linspace(0,12,13)*np.pi/18 # for 10 degrees phase diff per step
ax.set_xticks(np.round(xticks,1))
ax.xaxis.grid() #vertical lines
